# 인공신경망 딥러닝 모델 구현

## iris 데이터 불러오기

In [ ]:
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=1)

## 싸이킷런을 이용한 구현

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30), activation="logistic", solver="adam",
                    max_iter=1000) 

In [ ]:
mlp.fit(X_train, y_train)

In [ ]:
mlp.score(X_test, y_test)

In [ ]:
pred = mlp.predict(X_test)

In [ ]:
import pandas as pd
pd.crosstab(pred, y_test) #교차분류표
# pd.crosstab(np.argmax(pred, axis=1), np.argmax(y_test, axis=1)) #교차분류표

## 파이토치를 이용한 구현

In [ ]:
import torch
torch.__version__

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X, y = iris.data, iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
import torch.nn as nn
from torchsummary import summary
model = nn.Sequential(
    nn.Linear(4, 50),
    nn.Sigmoid(),
    nn.Linear(50, 30),
    nn.Sigmoid(),
    nn.Linear(30, 3),
    nn.Sigmoid()
)
summary(model, input_size=(1,4))

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.01)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
# 훈련 데이터 로더 정의
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# 테스트 데이터 로더 정의
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
def train(model, train_loader, criterion, optimizer, epochs=100):
    model.train()  # 모델을 학습 모드로 설정
    for epoch in range(epochs):
        running_loss = 0.0  # 현재 epoch의 누적 손실 초기화
        for data, target in train_loader:  # 미니배치 단위로 데이터 로드
            optimizer.zero_grad()  # 옵티마이저의 기울기 버퍼 초기화
            output = model(data)   # 모델을 통해 예측값 계산
            loss = criterion(output, target)  # 손실 계산
            loss.backward()  # 역전파를 통해 기울기 계산
            optimizer.step()  # 옵티마이저를 통해 가중치 업데이트
            running_loss += loss.item()  # 손실 누적
        # epoch 종료 후 평균 손실 출력
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# 모델 학습
train(model, train_loader, criterion, optimizer)

In [ ]:
# 예측 함수 정의
def predict(model, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    predictions = []  # 예측값을 저장할 리스트 초기화
    with torch.no_grad():  # 기울기 계산 비활성화
        for data in test_loader:  # 미니배치 단위로 데이터 로드
            inputs = data[0]  # 텐서 데이터셋에서 입력 데이터 추출
            output = model(inputs)  # 모델을 통해 예측값 계산
            predicted_classes = output.argmax(dim=1, keepdim=True)  # 가장 높은 값을 가진 클래스 선택
            predictions.append(predicted_classes)  # 예측값 리스트에 추가
    return torch.cat(predictions).numpy()  # 리스트를 텐서로 변환하고 numpy 배열로 반환

# 모델 예측
predictions = predict(model, test_loader)

In [ ]:
print(predictions.ravel())

In [ ]:
import pandas as pd
pd.crosstab(y_test, predictions.ravel())

## 케라스를 이용한 구현

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras import Sequential, layers
model = Sequential([
    layers.Input(shape=(4,)),
    layers.Dense(50, activation='sigmoid'),
    layers.Dense(30, activation='sigmoid'),
    layers.Dense(3, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=300, verbose=1)

In [ ]:
model.evaluate(X_test, y_test) # 출력값은 loss와 accuracy

## 파이토치를 이용한 구현

In [ ]:
# # CPU 버전 설치 (최신 버전)
# ! pip install torch

# # GPU 지원 버전 설치 (최신 버전)
# ! pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
print(torch.__version__)

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [ ]:
import torch
import torch.nn as nn

# 모델 클래스 정의
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 3)
        self.softmax = nn.Softmax(dim=1)
        # 가중치 초기화 (Xavier 초기화)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# 모델 생성
model = CustomModel()

In [ ]:
# 모델 정의를 아래처럼 Sequential을 이용할 수 있음
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(4, 50), nn.Sigmoid(),
    nn.Linear(50, 30), nn.Sigmoid(),
    nn.Linear(30, 3), nn.Softmax(dim=1)
)

In [ ]:
# 손실 함수 및 옵티마이저 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# 모델 학습
for epoch in range(1000):
    inputs = torch.from_numpy(X_train).float()
    labels = torch.from_numpy(y_train).long()
    
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

In [ ]:
# 모델 평가
with torch.no_grad():
    inputs = torch.from_numpy(X_test).float()
    labels = torch.from_numpy(y_test).long()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == labels).sum().item() / len(y_test)

print(f"Test Loss:{loss.item()}, Test Accuracy:{accuracy}")

### 파이토치의 Sequential 이용

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

In [ ]:
# 넘파이 배열을 토치 텐서로 변환
import torch

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 모델 정의
model = nn.Sequential(
    nn.Linear(4, 50), nn.Sigmoid(),
    nn.Linear(50, 30), nn.Sigmoid(),
    nn.Linear(30, 3), nn.Softmax(dim=1)
)

# 가중치 초깃값 지정
for layer in model:
    if isinstance(layer, nn.Linear):
        nn.init.xavier_uniform_(layer.weight)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# 모델 학습
for epoch in range(300):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 모델 평가
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print("Loss: {}, Accuracy: {}".format(loss.item(), accuracy))